In [1]:
# Configuración básica del proyecto
PROJECT_NAME = 'Feed Forward Neural Network'

import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from classes.DataHandler import DataHandler
from classes.FeedForwardNetwork_opt import FeedForwardNetwork_opt

dh = DataHandler()
DEVICE = 'cuda'
dh.DEVICE = DEVICE

dh.set_targets(['Mo_cumulative'])
table_unified, vars_desc = dh.load_data()
inputs, targets = dh.preprocess_data(vars_desc, table_unified)
X, y, scales = dh.normalize_data(inputs, targets)

# Convertir X y y a tensores de PyTorch y moverlos a la GPU
X = torch.tensor(X, dtype=torch.float32).to(dh.DEVICE)
y = torch.tensor(y, dtype=torch.float32).to(dh.DEVICE)

# Dividir los datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(
    X.cpu().numpy(), y.cpu().numpy(), test_size=0.2, random_state=42
)

# Convertir los conjuntos a tensores y moverlos a la GPU
X_train = torch.tensor(X_train, dtype=torch.float32).to(dh.DEVICE)
X_val = torch.tensor(X_val, dtype=torch.float32).to(dh.DEVICE)
y_train = torch.tensor(y_train, dtype=torch.float32).to(dh.DEVICE)
y_val = torch.tensor(y_val, dtype=torch.float32).to(dh.DEVICE)

input_dim = X_train.shape[1]
hidden_dim = 528
show_info = 'True'

ffn = FeedForwardNetwork_opt(input_dim, hidden_dim, DEVICE, show_info=show_info)
model = ffn.train_model(X_train, y_train, X_val, y_val, num_epochs=5000, lr=0.001)

2025-05-17 12:44:04,289 - INFO - Configuración de logging inicializada correctamente.
2025-05-17 12:44:19,788 - INFO - Columnas objetivo actualizadas: ['Mo_cumulative']
2025-05-17 12:44:19,788 - INFO - Inicio de la carga de datos.
2025-05-17 12:44:19,793 - INFO - Tabla Tabla_Unificada cargada correctamente.
2025-05-17 12:44:19,795 - INFO - Tabla Variables_Description cargada correctamente.
2025-05-17 12:44:19,799 - INFO - Conexión cerrada.
2025-05-17 12:44:19,788 - INFO - Columnas objetivo actualizadas: ['Mo_cumulative']
2025-05-17 12:44:19,788 - INFO - Inicio de la carga de datos.
2025-05-17 12:44:19,793 - INFO - Tabla Tabla_Unificada cargada correctamente.
2025-05-17 12:44:19,795 - INFO - Tabla Variables_Description cargada correctamente.
2025-05-17 12:44:19,799 - INFO - Conexión cerrada.
2025-05-17 12:44:19,800 - INFO - Datos cargados exitosamente.
2025-05-17 12:44:19,800 - INFO - Inicio del preprocesamiento de datos.
2025-05-17 12:44:19,811 - INFO - Variables de entrada procesadas:

🔗 Conexión establecida con data/Tronaduras_vs_Sismicidad.db
✅ Datos leídos desde 'Processed_Data.Tabla_Unificada' y convertidos a DataFrame.
✅ Datos leídos desde 'Raw_Data.Variables_Description' y convertidos a DataFrame.
🔌 Conexión cerrada manualmente.
[2025-05-17 12:44:20] 🚀 [FeedForward] Entrenando red...
[2025-05-17 12:44:20] ✨ [FeedForward] Mejor modelo en época 0 📉 train loss: 0.1920 📊 val loss: 0.0255
[2025-05-17 12:44:20] 🔄 [FeedForward] [Época 0] 🧠 Pérdida entrenamiento: 0.1920 | Validación: 0.0255
[2025-05-17 12:44:20] ✨ [FeedForward] Mejor modelo en época 4 📉 train loss: 0.0256 📊 val loss: 0.0178
[2025-05-17 12:44:20] ✨ [FeedForward] Mejor modelo en época 10 📉 train loss: 0.0226 📊 val loss: 0.0166
[2025-05-17 12:44:20] ✨ [FeedForward] Mejor modelo en época 11 📉 train loss: 0.0174 📊 val loss: 0.0164
[2025-05-17 12:44:20] ✨ [FeedForward] Mejor modelo en época 18 📉 train loss: 0.0161 📊 val loss: 0.0162
[2025-05-17 12:44:20] ✨ [FeedForward] Mejor modelo en época 19 📉 train loss:

In [3]:
# Predicción y desnormalización
model.eval()
with torch.no_grad():
    y_pred_train = model(X_train).squeeze().cpu().numpy()
    y_pred_val = model(X_val).squeeze().cpu().numpy()

y_train_np = y_train.cpu().numpy()
y_val_np = y_val.cpu().numpy()
y_train_mo = dh.denormalize_value(scales, y_train_np, 'Mo_cumulative')
y_pred_train_mo = dh.denormalize_value(scales, y_pred_train, 'Mo_cumulative')
y_val_mo = dh.denormalize_value(scales, y_val_np, 'Mo_cumulative')
y_pred_val_mo = dh.denormalize_value(scales, y_pred_val, 'Mo_cumulative')

# Graficar resultados de entrenamiento
dh.plot_results(
    (y_train_mo, 'Valores Reales'),
    (y_pred_train_mo, 'Predicciones FFN'),
    title='Predicciones FeedForward vs Reales (Entrenamiento)',
    size=(1200, 400)
)

# Graficar resultados de validación
dh.plot_results(
    (y_val_mo, 'Valores Reales'),
    (y_pred_val_mo, 'Predicciones FFN'),
    title='Predicciones FeedForward vs Reales (Validación)',
    size=(1200, 400)
)

In [4]:
# Evaluación
dh.evaluar_metricas(y_train_mo, y_pred_train_mo, 'FeedForward - Entrenamiento')
dh.evaluar_metricas(y_val_mo, y_pred_val_mo, 'FeedForward - Validación')


📊 Métricas para FeedForward - Entrenamiento:
   MAE  : 0.0000
   RMSE : 0.0001
   R²   : 1.0000

📊 Métricas para FeedForward - Validación:
   MAE  : 0.2967
   RMSE : 0.3779
   R²   : 0.1353


{'MAE': 0.2967330880270212,
 'RMSE': np.float64(0.3778763264632582),
 'R2': 0.13529757144740073}

In [5]:
# Gráficos de regresión
dh.plot_regression(y_train_mo, y_pred_train_mo, title='Entrenamiento FeedForward', size=(1200, 400))
dh.plot_regression(y_val_mo, y_pred_val_mo, title='Validación FeedForward', size=(1200, 400))